In [3]:
from collections import defaultdict
import csv 
from csv import reader
from scipy.optimize import linear_sum_assignment
#from PIL import Image as im
from IPython.core.display import Image, display
import requests
from io import BytesIO

'''
Ambiguity
    Multiple Options > A/M
    Definition, Location > A/L
    Definition, Destination/Intention > A/D
    Definition, Kind/Type > A/K
    Definition, Time > A/T
    Gouping > A/G

Uncertainty > U

Mistakes/Bad Data
    Annotator mistakes > M/A
    Bad question/bad images > M/B

Misc/need reclassifying > ~

Question with multiple classification will have '.' between the classificiations
ex: A/M.A/L.M/B)
'''

input_csv = "csv_results_ambiguity.csv"
data = []
with open(input_csv) as read_obj:
    csv_reader = csv.DictReader(read_obj)
    for row in csv_reader:
        data.append(row)


In [5]:
'''
Display by classification
'''
search_item = 'A/L'

for row in data:
    temp = row['Answer.skip_reason'].strip('"')
    labels = temp.split('.')
    
    if search_item in labels:
        print(row['Input.questionStr'])
        print(row['Input.imgUrl'])
        print(labels)
        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))
       

"Where was the picture taken?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000238006.jpg"
['A/L']


"Where are the pizza?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000448944.jpg"
['A/L']


"Where is the fan?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000418141.jpg"
['A/L']


"Where are the passengers?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000170857.jpg"
['A/M', 'A/L']


"Where are the people sitting?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000426980.jpg"
['A/L']


...

"Where is the pedestrian light?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000318533.jpg"
['A/L']


"Where is the bank of America advertising?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000129672.jpg"
['A/L']


"Where is the person eating?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000215863.jpg"
['A/L']


"Where is the stop sign?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000305538.jpg"
['A/L']


"Where are the flowers?"
"https://cs.jhu.edu/~esteng/images_for_hit/COCO_train2014_000000463039.jpg"
['A/L']


In [ ]:
'''
Display by number of classifications
'''
search_num = 3

for row in data:
    temp = row['Answer.skip_reason'].strip('"')
    labels = temp.split('.')
    
    if len(labels) == search_num:
        print(row['Input.questionStr'])
        print(labels)
        image_url = row['Input.imgUrl'].strip('"')
        display(Image(url=image_url))